# Libraries

In [31]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
### plotting
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import cvxpy as cp
import time
### Other
import itertools
from timeit import default_timer as timer
from itertools import product
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings("ignore")

In [32]:
### QMC
!pip install gpytorch
!pip install qmcpy
import qmcpy

In [48]:
def GBM_characteristic_function(u, sigma, SIGMA, T, r):
    """Calculate the extended characteristic function of Multivariate GBM.
    Args:
    - u (array): Array of Fourier frequencies.
    - sigma (array): Array of volatilities of each stock.
    - SIGMA (array): Covariance matrix.
    - T (float): Time to maturity.
    - r (float): Risk-free interest rate.
    Returns:
    - phi (complex): Extended characteristic function value.
    """
    d = len(sigma)  # number of stocks
    phi = np.exp(np.dot(np.multiply(1j * T, u), r * np.ones(d) - 0.5 * np.diag(SIGMA)) - 0.5 * T * np.dot(u, np.dot(SIGMA, u)))
    return phi

def covariance_matrix(sigma, rho):
    """Compute the covariance matrix.
    Args:
    - sigma (array): Array of volatilities of each stock.
    - rho (array): Correlation matrix.
    Returns:
    - SIGMA (array): Covariance matrix.
    """
    SIGMA = np.dot(np.diag(sigma), np.dot(rho, np.diag(sigma)))
    return SIGMA

def fourier_payoff_call_on_min(u):
    """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
    Args:
    - u (array): Array of Fourier frequencies.
    Returns:
    - payoff (float): Call on min option payoff Fourier transofrm value.
    """
    denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
    return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_GBM_call_on_min(R):
    """Calculate the integrand of the GBM to optimize .
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): integrand value at the origin (u = 0)
    """
    d = len(S0)  # dimensionality
    X0 = np.log(np.divide(S0, K))
    y = np.multiply(1j, R)
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
    p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
    discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    integrand = discount * phi * p
    return np.real(integrand)  # Real part of the integrand


def integrand_to_optimize_GBM_basket_put(R):
    """Calculate the integrand of the GBM to optimize .
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): integrand value at the origin (u = 0)
    """
    d = len(S0)  # dimensionality
    X0 = np.log(np.divide(S0, d*K))
    y = np.multiply(1j, R)
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
    p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
    discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    integrand = discount * phi * p
    return np.real(integrand)  # Real part of the integrand


def RQMC_fourier_GBM_call_on_min_pricer(S0, K, r, T, sigma, SIGMA, N, R, m, SIGMA_IS):
    """Perform QMC estimate for GBM call option pricing.
    Args:
    - S0 (array): Initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - sigma (array): Array of volatilities of each stock.
    - SIGMA (array): Covariance matrix.
    - N (int): Number of QMC points.
    - m (int): Number of randomizations of RQMC (digital shifts).
    - R (array): Array of damping parameters.
    - SIGMA_IS (array): Covariance matrix of the domain transformation PDF.

    Returns:
    - qmc_estimate (float): RQMC estimate of option price.
    - qmc_stat_error (float): RQMC statistical error (absolute not relative).
    """
    L_IS = la.cholesky(SIGMA_IS)
    d = len(S0)  # number of stocks
    X0 = np.log(np.divide(S0, K))
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    V_list = np.zeros(m)  # Contains m price estimates each corresponding to a shifted sobol sequence
    for i in range(m):
        xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=i).gen_samples(N)
        xi_sobol_shifted_mapped = stats.norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)
        V = 0  # Initialization of Option price contract value for ith rQMC iteration
        for n in range(N):  # for each sobol point
            u = L_IS @ xi_sobol_shifted_mapped[n]  # inverse CDF of uniformly distributed sample to have sample from standard normal
            y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter to integrate over axis parallel to real line.
            phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # evaluation of characteristic function at all sobol points
            p = fourier_payoff_call_on_min(y)  # evaluation of Fourier Transformed Payoff function at a sobol point
            IS_pdf_prod = stats.multivariate_normal.pdf(x=u, mean=np.zeros(d), cov=SIGMA_IS)  # product of gaussian densities evaluated at each component of d-dim vector: {rho(u_i)}i=1..d
            V += (1 / N) * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
        V_list[i] = K * discount * np.real(V)
    qmc_estimate = np.mean(V_list)  # RQMC estimate
    qmc_stat_error = 1.96 * np.std(V_list) / np.sqrt(m)  # RQMC statistical error.
    return qmc_estimate, qmc_stat_error


def RQMC_fourier_GBM_basket_put_pricer(S0, K, r, T, sigma, SIGMA, N, R, m, SIGMA_IS):
    """Perform QMC estimate for GBM call option pricing.
    Args:
    - S0 (array): Initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - sigma (array): Array of volatilities of each stock.
    - SIGMA (array): Covariance matrix.
    - N (int): Number of QMC points.
    - m (int): Number of randomizations of RQMC (digital shifts).
    - R (array): Array of damping parameters.
    - SIGMA_IS (array): Covariance matrix of the domain transformation PDF.

    Returns:
    - qmc_estimate (float): RQMC estimate of option price.
    - qmc_stat_error (float): RQMC statistical error (absolute not relative).
    """
    L_IS = la.cholesky(SIGMA_IS)
    d = len(S0)  # number of stocks
    X0 = np.log(np.divide(S0, d*K))
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    V_list = np.zeros(m)  # Contains m price estimates each corresponding to a shifted sobol sequence
    for i in range(m):
        xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=i).gen_samples(N)
        xi_sobol_shifted_mapped = stats.norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)
        V = 0  # Initialization of Option price contract value for ith rQMC iteration
        for n in range(N):  # for each sobol point
            u = L_IS @ xi_sobol_shifted_mapped[n]  # inverse CDF of uniformly distributed sample to have sample from standard normal
            y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter to integrate over axis parallel to real line.
            phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # evaluation of characteristic function at all sobol points
            p = fourier_payoff_basket_put(y)  # evaluation of Fourier Transformed Payoff function at a sobol point
            IS_pdf_prod = stats.multivariate_normal.pdf(x=u, mean=np.zeros(d), cov=SIGMA_IS)  # product of gaussian densities evaluated at each component of d-dim vector: {rho(u_i)}i=1..d
            V += (1 / N) * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
        V_list[i] = K * discount * np.real(V)
    qmc_estimate = np.mean(V_list)  # RQMC estimate
    qmc_stat_error = 1.96 * np.std(V_list) / np.sqrt(m)  # RQMC statistical error.
    return qmc_estimate, qmc_stat_error

# Call on min Options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [ ]:
# Model and payoff parameters
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 6 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R
def rainbow_constraint_2(R):
  return -1 - np.sum(R)
cons = ( {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_call_on_min, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-4.67675337 -4.67675337 -4.67675337 -4.67675337 -4.67675335 -4.67675335]


## Pricing using RQMC in the Fourier space

In [34]:
############### Model and payoff parameters ###############
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 6 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
m = 30 # number of digital shifts of RQMC
N = 2**6 # number of QMC Sobol points
SIGMA_IS = (1 / T) * la.inv(SIGMA) # Proposed QMC domain transformation
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
RQMC_estimate, RQMC_stat_estimate = RQMC_fourier_GBM_call_on_min_pricer(S0,K,r,T,sigma,SIGMA,N,R,m,SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5), ", Relative Statistical Error =", round(RQMC_stat_estimate / RQMC_estimate,5)  )

RQMC estimate = 0.03378 , Relative Statistical Error = 0.00494


# Basket Put

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [57]:
 # Model and payoff parameters
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 4 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': basket_put_constraint},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [3.51974639 3.51974639 3.51974638 3.51974638]


## Pricing using RQMC in the Fourier space

In [58]:
############### Model and payoff parameters ###############
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 4 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
m = 30 # number of digital shifts of RQMC
N = 2**6 # number of QMC Sobol points
SIGMA_IS = (1 / T) * la.inv(SIGMA) # Proposed QMC domain transformation
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
RQMC_estimate, RQMC_stat_estimate = RQMC_fourier_GBM_basket_put_pricer(S0,K,r,T,sigma,SIGMA,N,R,m,SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5), ", Relative Statistical Error =", round(RQMC_stat_estimate / RQMC_estimate,5)  )

RQMC estimate = 3.98108 , Relative Statistical Error = 0.06247
